In [ ]:
import pandas as pd
from itertools import islice
import random
import numpy as np
import json
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit

seed = 42
random.seed(seed)
np.random.seed(seed)

import seaborn as sn
sn.set_theme(style="white", palette="rocket_r")

Mounted at /content/gdrive


In [ ]:
labels = pd.read_csv('preprocessed_labels.csv', index_col='patient')

In [ ]:
columns_that_matter = ['unitdischargeoffset', 'predictedhospitalmortality', 'actualhospitalmortality']
labels = labels[columns_that_matter]

In [ ]:
flats = pd.read_csv('preprocessed_flat.csv', index_col='patient')

In [ ]:
flats = flats[flats['> 89'] != 1]

In [ ]:
flats = flats.drop(['nullweight', 'nullheight', '> 89'], axis=1)

In [ ]:
# Load the diagnosis and post_diagnosis files
diagnoses = pd.read_csv('preprocessed_diagnoses.csv', index_col='patient')
post_diagnoses = pd.read_csv('preprocessed_diagnoses_post.csv', index_col='patient')

In [ ]:
def diagnosis_processing_MI(diagnoses, post_diagnoses):
    # Relevant diagnoses
    list_diagnoses = ['cardiovascular|chest pain / ASHD|acute coronary syndrome|acute myocardial infarction (no ST elevation)',
                     'cardiovascular|chest pain / ASHD|acute coronary syndrome|acute myocardial infarction (with ST elevation)',
                     'cardiovascular|chest pain / ASHD|acute coronary syndrome|s/p PTCA',
                     'Acute MI location', 'Acute MI location|inferior', 'Acute MI location|non-Q',
                     'Non-operative|Diagnosis|Cardiovascular|Infarction, acute myocardial (MI)',
                     'Cardiovascular (R)|Myocardial Infarction', 'Cardiovascular (R)|Myocardial Infarction|MI - date unknown',
                     'Cardiovascular (R)|Myocardial Infarction|MI - remote', 'Cardiovascular (R)|Myocardial Infarction|MI - within 6 months']

    # Create MI feature
    diagnoses[list_diagnoses] = diagnoses[list_diagnoses].astype('bool')
    i = 0
    for col in list_diagnoses:
        if i == 0:
            MI_column = diagnoses[col]
        MI_column = MI_column | diagnoses[col]
        i += 1

    MI_column = MI_column.astype(float)
    diagnoses = diagnoses.assign(MI=MI_column)
    diagnoses[list_diagnoses] = diagnoses[list_diagnoses].astype('float')

    # Relevant diagnoses
    list_diagnoses = [
                      'cardiovascular|arrhythmias',
         'cardiovascular|arrhythmias|bradycardia',
 'cardiovascular|arrhythmias|sinus tachycardia',
 'cardiovascular|arrhythmias|ventricular tachycardia'
    ]
    # Create MI feature
    post_diagnoses[list_diagnoses] = post_diagnoses[list_diagnoses].astype('bool')
    i = 0
    for col in list_diagnoses:
        if i == 0:
            MI_column = post_diagnoses[col]
        MI_column = MI_column | post_diagnoses[col]
        i += 1

    MI_column = MI_column.astype(float)
    post_diagnoses = post_diagnoses.assign(ARR=MI_column)
    post_diagnoses[list_diagnoses] = post_diagnoses[list_diagnoses].astype('float')

    merged_diagnosis = post_diagnoses['ARR'].to_frame().merge(diagnoses, left_index=True, right_index=True)
    merged_diagnosis['ARR'] = merged_diagnosis['ARR'].astype(int)

       # Relevant diagnoses
    list_diagnoses = [
         'cardiovascular|arrhythmias|atrial fibrillation',
 'cardiovascular|arrhythmias|atrial fibrillation|with controlled ventricular response',
 'cardiovascular|arrhythmias|atrial fibrillation|with rapid ventricular response'
    ]
    # Create MI feature
    post_diagnoses[list_diagnoses] = post_diagnoses[list_diagnoses].astype('bool')
    i = 0
    for col in list_diagnoses:
        if i == 0:
            MI_column = post_diagnoses[col]
        MI_column = MI_column | post_diagnoses[col]
        i += 1

    MI_column = MI_column.astype(float)
    post_diagnoses = post_diagnoses.assign(AF=MI_column)
    post_diagnoses[list_diagnoses] = post_diagnoses[list_diagnoses].astype('float')

    merged_diagnosis = post_diagnoses['AF'].to_frame().merge(merged_diagnosis, left_index=True, right_index=True)
    merged_diagnosis['AF'] = merged_diagnosis['AF'].astype(int)

    # Relevant diagnoses
    list_diagnoses = [
        'cardiovascular|ventricular disorders|congestive heart failure',
                      'cardiovascular|shock / hypotension|cardiogenic shock',
 'cardiovascular|cardiac arrest',
 'cardiovascular|cardiac arrest|cardiac arrest|witnessed, < 15 minutes CPR'
    ]
    # Create MI feature
    post_diagnoses[list_diagnoses] = post_diagnoses[list_diagnoses].astype('bool')
    i = 0
    for col in list_diagnoses:
        if i == 0:
            MI_column = post_diagnoses[col]
        MI_column = MI_column | post_diagnoses[col]
        i += 1

    MI_column = MI_column.astype(float)
    post_diagnoses = post_diagnoses.assign(HF=MI_column)
    post_diagnoses[list_diagnoses] = post_diagnoses[list_diagnoses].astype('float')

    merged_diagnosis = post_diagnoses['HF'].to_frame().merge(merged_diagnosis, left_index=True, right_index=True)
    merged_diagnosis['HF'] = merged_diagnosis['HF'].astype(int)

    # Relevant diagnoses
    list_diagnoses = [
        'cardiovascular|vascular disorders|peripheral vascular ischemia'
    ]
    # Create MI feature
    post_diagnoses[list_diagnoses] = post_diagnoses[list_diagnoses].astype('bool')
    i = 0
    for col in list_diagnoses:
        if i == 0:
            MI_column = post_diagnoses[col]
        MI_column = MI_column | post_diagnoses[col]
        i += 1

    MI_column = MI_column.astype(float)
    post_diagnoses = post_diagnoses.assign(PAD=MI_column)
    post_diagnoses[list_diagnoses] = post_diagnoses[list_diagnoses].astype('float')

    merged_diagnosis = post_diagnoses['PAD'].to_frame().merge(merged_diagnosis, left_index=True, right_index=True)
    merged_diagnosis['PAD'] = merged_diagnosis['PAD'].astype(int)
    merged_diagnosis['MI'] = merged_diagnosis['MI'].astype(int)

    return merged_diagnosis[['MI', 'PAD', 'HF', 'AF', 'ARR']]

In [ ]:
merged_diagnosis = diagnosis_processing_MI(diagnoses, post_diagnoses)

In [ ]:
final = merged_diagnosis.merge(labels, left_index=True, right_index=True)

In [ ]:
# Keep only those patients that have had MI
final = final.drop(final[final.MI == 0].index)
final = final.drop(columns=['MI'])

In [ ]:
final = final.merge(flats, left_index=True, right_index=True)
final

,PAD,HF,AF,ARR,unitdischargeoffset,predictedhospitalmortality,actualhospitalmortality,gender,age,admissionheight,...,physicianspeciality_surgery-vascular,physicianspeciality_unknown,numbedscategory_100 - 249,numbedscategory_250 - 499,numbedscategory_<100,numbedscategory_>= 500,region_Midwest,region_Northeast,region_South,region_West
patient,,,,,,,,,,,,,,,,,,,,,
141203,0,0,0,0,1869,0.470973,0,0.0,77.0,160.0,...,0,0,1,0,0,0,1,0,0,0
141844,0,0,0,1,4110,0.001299,0,0.0,53.0,162.6,...,0,0,0,0,0,1,1,0,0,0
142173,0,0,0,0,37333,0.112978,0,0.0,59.0,162.6,...,0,0,0,0,0,0,1,0,0,0
143687,0,0,0,0,2594,0.017727,0,1.0,53.0,182.9,...,0,0,1,0,0,0,1,0,0,0
144108,0,0,1,1,10458,0.068377,0,1.0,55.0,170.2,...,0,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3352895,0,0,0,0,4172,0.225746,1,0.0,83.0,147.3,...,0,0,0,0,0,1,0,0,1,0
3352914,0,0,0,0,982,0.020733,0,1.0,68.0,180.3,...,0,0,0,0,0,1,0,0,1,0
3352962,0,0,0,0,2241,0.030529,0,1.0,60.0,182.9,...,0,0,0,0,0,1,0,0,1,0


In [ ]:
final.columns

Index(['PAD', 'HF', 'AF', 'ARR', 'unitdischargeoffset',
       'predictedhospitalmortality', 'actualhospitalmortality', 'gender',
       'age', 'admissionheight', 'admissionweight', 'hour', 'teachingstatus',
       'ethnicity_African American', 'ethnicity_Asian', 'ethnicity_Caucasian',
       'ethnicity_Hispanic', 'ethnicity_Native American',
       'ethnicity_Other/Unknown', 'unittype_CCU-CTICU', 'unittype_CSICU',
       'unittype_CTICU', 'unittype_Cardiac ICU', 'unittype_MICU',
       'unittype_Med-Surg ICU', 'unittype_Neuro ICU', 'unittype_SICU',
       'unitadmitsource_Acute Care/Floor', 'unitadmitsource_Direct Admit',
       'unitadmitsource_Emergency Department', 'unitadmitsource_Floor',
       'unitadmitsource_Operating Room', 'unitadmitsource_Other Hospital',
       'unitadmitsource_PACU', 'unitadmitsource_Recovery Room',
       'unitadmitsource_Step-Down Unit (SDU)', 'unitadmitsource_misc',
       'unitvisitnumber_1', 'unitvisitnumber_2', 'unitvisitnumber_3',
       'unitvisit

In [ ]:
# Keep only those patients that have had CCU stays
final = final.loc[(final['unittype_CCU-CTICU'] == 1) | (final['unittype_CSICU'] == 1) | (final['unittype_CTICU'] == 1) | (final['unittype_Cardiac ICU'] == 1)]

In [ ]:
final

,PAD,HF,AF,ARR,unitdischargeoffset,predictedhospitalmortality,actualhospitalmortality,gender,age,admissionheight,...,physicianspeciality_surgery-vascular,physicianspeciality_unknown,numbedscategory_100 - 249,numbedscategory_250 - 499,numbedscategory_<100,numbedscategory_>= 500,region_Midwest,region_Northeast,region_South,region_West
patient,,,,,,,,,,,,,,,,,,,,,
141844,0,0,0,1,4110,0.001299,0,0.0,53.0,162.6,...,0,0,0,0,0,1,1,0,0,0
144989,0,0,0,0,2760,0.092861,0,0.0,78.0,158.0,...,0,0,0,0,0,1,1,0,0,0
145049,0,0,0,0,1266,0.022707,0,1.0,60.0,185.4,...,0,0,0,0,0,1,1,0,0,0
146512,0,1,0,1,11270,-1.000000,0,0.0,62.0,170.2,...,0,0,0,0,0,1,1,0,0,0
146619,0,1,0,1,7110,-1.000000,0,1.0,74.0,182.8,...,0,0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3352721,0,0,0,0,23834,0.636440,0,0.0,88.0,157.5,...,0,0,0,0,0,1,0,0,1,0
3352781,0,0,0,0,1658,0.022316,0,1.0,60.0,170.2,...,0,0,0,0,0,1,0,0,1,0
3352914,0,0,0,0,982,0.020733,0,1.0,68.0,180.3,...,0,0,0,0,0,1,0,0,1,0


In [ ]:
timeseries_lab = pd.read_csv('timeseries_lab.csv', index_col=['patient', 'time'])

In [ ]:
timeseries_lab

-basos  -eos  -lymphs  -monos  -polys  ALT (SGPT)  \
patient time                                                                   
141168  0 days 03:51:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 08:36:00       0.0   1.0     19.0    19.0    61.0        40.0   
        0 days 18:53:00       0.0   0.0      6.0    14.0    80.0       358.0   
        1 days 06:05:00       NaN   NaN      NaN     NaN     NaN         NaN   
        1 days 09:30:00       NaN   NaN      NaN     NaN     NaN         NaN   
...                           ...   ...      ...     ...     ...         ...   
3353254 0 days 03:31:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 06:34:00       NaN   NaN      NaN     NaN     NaN         NaN   
3353263 -1 days +23:23:00     NaN   NaN      NaN     NaN     NaN         NaN   
        -1 days +23:53:00     0.0   1.0     24.0    10.0     NaN         NaN   
        0 days 11:02:00       NaN   NaN      NaN     NaN     NaN         NaN   

                           AST (SGOT)   BUN  Base Excess   FiO2  ...  paCO2  \
patient time                                                     ...          
141168  0 days 03:51:00           NaN   NaN          NaN    NaN  ...    NaN   
        0 days 08:36:00          59.0  26.0          NaN    NaN  ...    NaN   
        0 days 18:53:00         878.0  27.0          NaN    NaN  ...    NaN   
        1 days 06:05:00           NaN   NaN          NaN   28.0  ...   46.0   
        1 days 09:30:00           NaN   NaN          NaN  100.0  ...   44.0   
...                               ...   ...          ...    ...  ...    ...   
3353254 0 days 03:31:00           NaN   NaN          NaN    NaN  ...    NaN   
        0 days 06:34:00           NaN   NaN          NaN    NaN  ...    NaN   
3353263 -1 days +23:23:00         NaN   NaN          NaN    NaN  ...    NaN   
        -1 days +23:53:00         NaN  13.0          NaN    NaN  ...    NaN   
        0 days 11:02:00           NaN   NaN          NaN    NaN  ...    NaN   

                           paO2  phosphate  platelets x 1000  potassium  \
patient time                                                              
141168  0 days 03:51:00     NaN        NaN               NaN        NaN   
        0 days 08:36:00     NaN        NaN             209.0        4.0   
        0 days 18:53:00     NaN        NaN             213.0        4.2   
        1 days 06:05:00    41.0        NaN               NaN        NaN   
        1 days 09:30:00    42.0        NaN               NaN        NaN   
...                         ...        ...               ...        ...   
3353254 0 days 03:31:00     NaN        NaN               NaN        NaN   
        0 days 06:34:00     NaN        NaN               NaN        NaN   
3353263 -1 days +23:23:00   NaN        NaN               NaN        NaN   
        -1 days +23:53:00   NaN        2.5             162.5        4.1   
        0 days 11:02:00     NaN        NaN               NaN        NaN   

                           sodium  total bilirubin  total protein  \
patient time                                                        
141168  0 days 03:51:00       NaN              NaN            NaN   
        0 days 08:36:00     139.0              2.6            7.1   
        0 days 18:53:00     139.0              4.1            7.1   
        1 days 06:05:00       NaN              NaN            NaN   
        1 days 09:30:00       NaN              NaN            NaN   
...                           ...              ...            ...   
3353254 0 days 03:31:00       NaN              NaN            NaN   
        0 days 06:34:00       NaN              NaN            NaN   
3353263 -1 days +23:23:00     NaN              NaN            NaN   
        -1 days +23:53:00   135.0              NaN            NaN   
        0 days 11:02:00       NaN              NaN            NaN   

                           troponin - I  urinary specific gravity  
patient time    

In [ ]:
# take the mean of any duplicate index entries for unstacking
timeseries_lab = timeseries_lab.groupby(level=[0, 1]).mean()

# Round up the time-stamps to the next hour
timeseries_lab.reset_index(level=1, inplace=True)
start = pd.to_datetime('2000-01-01 00:00:00')
timeseries_lab.time = pd.to_timedelta(timeseries_lab.time, errors='coerce') + start
timeseries_lab.time = timeseries_lab.time.dt.ceil(freq='H')
timeseries_lab.time = timeseries_lab.time - start
timeseries_lab.time = pd.to_timedelta(timeseries_lab.time, unit='T')
timeseries_lab.set_index('time', append=True, inplace=True)
timeseries_lab.reset_index(level=0, inplace=True)
resampled = timeseries_lab.groupby('patient').resample('H', closed='right', label='right').mean().drop(columns='patient')

In [ ]:
resampled

-basos  -eos  -lymphs  -monos  -polys  ALT (SGPT)  \
patient time                                                                 
141168  0 days 04:00:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 05:00:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 06:00:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 07:00:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 08:00:00     NaN   NaN      NaN     NaN     NaN         NaN   
...                         ...   ...      ...     ...     ...         ...   
3353263 0 days 09:00:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 10:00:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 11:00:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 12:00:00     NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 13:00:00     NaN   NaN      NaN     NaN     NaN         NaN   

                         AST (SGOT)  BUN  Base Excess  FiO2  ...  paCO2  paO2  \
patient time                                                 ...                
141168  0 days 04:00:00         NaN  NaN          NaN   NaN  ...    NaN   NaN   
        0 days 05:00:00         NaN  NaN          NaN   NaN  ...    NaN   NaN   
        0 days 06:00:00         NaN  NaN          NaN   NaN  ...    NaN   NaN   
        0 days 07:00:00         NaN  NaN          NaN   NaN  ...    NaN   NaN   
        0 days 08:00:00         NaN  NaN          NaN   NaN  ...    NaN   NaN   
...                             ...  ...          ...   ...  ...    ...   ...   
3353263 0 days 09:00:00         NaN  NaN          NaN   NaN  ...    NaN   NaN   
        0 days 10:00:00         NaN  NaN          NaN   NaN  ...    NaN   NaN   
        0 days 11:00:00         NaN  NaN          NaN   NaN  ...    NaN   NaN   
        0 days 12:00:00         NaN  NaN          NaN   NaN  ...    NaN   NaN   
        0 days 13:00:00         NaN  NaN          NaN   NaN  ...    NaN   NaN   

                         phosphate  platelets x 1000  potassium  sodium  \
patient time                                                              
141168  0 days 04:00:00        NaN               NaN        NaN     NaN   
        0 days 05:00:00        NaN               NaN        NaN     NaN   
        0 days 06:00:00        NaN               NaN        NaN     NaN   
        0 days 07:00:00        NaN               NaN        NaN     NaN   
        0 days 08:00:00        NaN               NaN        NaN     NaN   
...                            ...               ...        ...     ...   
3353263 0 days 09:00:00        NaN               NaN        NaN     NaN   
        0 days 10:00:00        NaN               NaN        NaN     NaN   
        0 days 11:00:00        NaN               NaN        NaN     NaN   
        0 days 12:00:00        NaN               NaN        NaN     NaN   
        0 days 13:00:00        NaN               NaN        NaN     NaN   

                         total bilirubin  total protein  troponin - I  \
patient time                                                            
141168  0 days 04:00:00              NaN            NaN           NaN   
        0 days 05:00:00              NaN            NaN           NaN   
        0 days 06:00:00              NaN            NaN           NaN   
        0 days 07:00:00              NaN            NaN           NaN   
        0 days 08:00:00              NaN            NaN           NaN   
...                                  ...            ...           ...   
3353263 0 days 09:00:00              NaN            NaN           NaN   
        0 days 10:00:00              NaN            NaN           NaN   
        0 days 11:00:00              NaN            NaN           NaN   
        0 days 12:00:00              NaN            NaN           NaN   
        0 days 13:00:00              NaN            NaN           NaN   

                         urina

In [ ]:
resampled.update(resampled.groupby(level=0).ffill())

In [ ]:
timeseries_lab = resampled

In [ ]:
timeseries_lab.reset_index(level=1, inplace=True)
timeseries_lab.time = pd.to_timedelta(timeseries_lab.time, errors='coerce')

# Convert the time-stamps into minutes
timeseries_lab.time = timeseries_lab.time.astype(int)/(1000000000*60)

timeseries_lab.reset_index(inplace=True)
timeseries_lab.set_index(['patient', 'time'], inplace=True)

In [ ]:
timeseries_lab

-basos  -eos  -lymphs  -monos  -polys  ALT (SGPT)  AST (SGOT)  \
patient time                                                                   
141168  240.0     NaN   NaN      NaN     NaN     NaN         NaN         NaN   
        300.0     NaN   NaN      NaN     NaN     NaN         NaN         NaN   
        360.0     NaN   NaN      NaN     NaN     NaN         NaN         NaN   
        420.0     NaN   NaN      NaN     NaN     NaN         NaN         NaN   
        480.0     NaN   NaN      NaN     NaN     NaN         NaN         NaN   
...               ...   ...      ...     ...     ...         ...         ...   
3353263 540.0     0.0   1.0     24.0    10.0     NaN         NaN         NaN   
        600.0     0.0   1.0     24.0    10.0     NaN         NaN         NaN   
        660.0     0.0   1.0     24.0    10.0     NaN         NaN         NaN   
        720.0     0.0   1.0     24.0    10.0     NaN         NaN         NaN   
        780.0     0.0   1.0     24.0    10.0     NaN         NaN         NaN   

                BUN  Base Excess  FiO2  ...  paCO2  paO2  phosphate  \
patient time                            ...                           
141168  240.0   NaN          NaN   NaN  ...    NaN   NaN        NaN   
        300.0   NaN          NaN   NaN  ...    NaN   NaN        NaN   
        360.0   NaN          NaN   NaN  ...    NaN   NaN        NaN   
        420.0   NaN          NaN   NaN  ...    NaN   NaN        NaN   
        480.0   NaN          NaN   NaN  ...    NaN   NaN        NaN   
...             ...          ...   ...  ...    ...   ...        ...   
3353263 540.0  13.0          NaN   NaN  ...    NaN   NaN        2.5   
        600.0  13.0          NaN   NaN  ...    NaN   NaN        2.5   
        660.0  13.0          NaN   NaN  ...    NaN   NaN        2.5   
        720.0  13.0          NaN   NaN  ...    NaN   NaN        2.5   
        780.0  13.0          NaN   NaN  ...    NaN   NaN        2.5   

               platelets x 1000  potassium  sodium  total bilirubin  \
patient time                                                          
141168  240.0               NaN        NaN     NaN              NaN   
        300.0               NaN        NaN     NaN              NaN   
        360.0               NaN        NaN     NaN              NaN   
        420.0               NaN        NaN     NaN              NaN   
        480.0               NaN        NaN     NaN              NaN   
...                         ...        ...     ...              ...   
3353263 540.0             162.5        4.1   135.0              NaN   
        600.0             162.5        4.1   135.0              NaN   
        660.0             162.5        4.1   135.0              NaN   
        720.0             162.5        4.1   135.0              NaN   
        780.0             162.5        4.1   135.0              NaN   

               total protein  troponin - I  urinary specific gravity  
patient time                                                          
141168  240.0            NaN           NaN                       NaN  
        300.0            NaN           NaN                       NaN  
        360.0            NaN           NaN                       NaN  
        420.0            NaN           NaN                       NaN  
        480.0            NaN           NaN                       NaN  
...                      ...           ...                       ...  
3353263 540.0            NaN           NaN                       NaN  
        600.0            NaN           NaN                       NaN  
        660.0            NaN           NaN                       NaN  
        720.0            NaN           NaN                       NaN  
        780.0            NaN           NaN                       NaN  

[10559534 rows x 46 columns]

In [ ]:
pd.set_option('display.max_columns', 1000)  # or 1000
pd.set_option('display.max_rows', 1000)  # or 1000
pd.set_option('display.max_colwidth', 1000)  # or 199

# Missing values per feature
timeseries_lab.isnull().sum(axis = 0)/10412737*100

-basos                      33.802938
-eos                        29.989089
-lymphs                     26.924785
-monos                      27.711485
-polys                      33.987222
ALT (SGPT)                  29.987380
AST (SGOT)                  29.233131
BUN                          5.427814
Base Excess                 50.143656
FiO2                        43.093454
HCO3                        42.950110
Hct                          4.906760
Hgb                          5.054406
MCH                         12.028336
MCHC                         8.010392
MCV                          7.998531
MPV                         34.484094
O2 Sat (%)                  48.858096
PT                          34.954585
PT - INR                    33.039911
PTT                         48.192488
RBC                          5.931812
RDW                         11.776894
WBC x 1000                   5.706223
albumin                     27.908580
alkaline phos.              30.670678
anion gap   

In [ ]:
timeseries_lab = timeseries_lab.drop(['urinary specific gravity', 'troponin - I', 'total bilirubin', 'total protein', 'pH',
                                      'paCO2', 'paO2', 'phosphate', 'PTT', 'O2 Sat (%)', 'HCO3', 'FiO2', 'Base Excess',
                                      '-basos', '-eos', '-lymphs', '-monos', '-polys', 'ALT (SGPT)', 'AST (SGOT)',
                                      'MPV', 'PT', 'PT - INR', 'lactate', 'bedside glucose', 'magnesium' ,'albumin',
                                      'alkaline phos.', 'anion gap', 'MCH','RDW', 'bicarbonate'], axis=1)

In [ ]:
# Drop missing values samples
missing_samples = timeseries_lab[timeseries_lab.isnull().any(axis=1)].index.get_level_values(0).tolist()
timeseries_lab.drop(missing_samples, level=0, axis=0, inplace=True)

In [ ]:
timeseries_lab

BUN   Hct    Hgb  MCHC   MCV    RBC  WBC x 1000  calcium  \
patient time                                                                
141194  -900.0  41.0  29.2   9.80  33.6  80.2  3.640       11.80      9.9   
        -840.0  41.0  29.2   9.80  33.6  80.2  3.640       11.80      9.9   
        -780.0  41.0  29.2   9.80  33.6  80.2  3.640       11.80      9.9   
        -720.0  41.0  29.2   9.80  33.6  80.2  3.640       11.80      9.9   
        -660.0  41.0  29.2   9.80  33.6  80.2  3.640       11.80      9.9   
...              ...   ...    ...   ...   ...    ...         ...      ...   
3353263  540.0  13.0  44.5  14.85  33.0  82.0  5.455        6.45      9.5   
         600.0  13.0  44.5  14.85  33.0  82.0  5.455        6.45      9.5   
         660.0  13.0  44.5  14.85  33.0  82.0  5.455        6.45      9.5   
         720.0  13.0  44.5  14.85  33.0  82.0  5.455        6.45      9.5   
         780.0  13.0  44.5  14.85  33.0  82.0  5.455        6.45      9.5   

                chloride  creatinine  glucose  platelets x 1000  potassium  \
patient time                                                                 
141194  -900.0     102.0        2.94    138.0             298.0        4.6   
        -840.0     102.0        2.94    138.0             298.0        4.6   
        -780.0     102.0        2.94    138.0             298.0        4.6   
        -720.0     102.0        2.94    138.0             298.0        4.6   
        -660.0     102.0        2.94    138.0             298.0        4.6   
...                  ...         ...      ...               ...        ...   
3353263  540.0     101.0        1.06    100.0             162.5        4.1   
         600.0     101.0        1.06    100.0             162.5        4.1   
         660.0     101.0        1.06    100.0             162.5        4.1   
         720.0     101.0        1.06    100.0             162.5        4.1   
         780.0     101.0        1.06    100.0             162.5        4.1   

                sodium  
patient time            
141194  -900.0   134.0  
        -840.0   134.0  
        -780.0   134.0  
        -720.0   134.0  
        -660.0   134.0  
...                ...  
3353263  540.0   135.0  
         600.0   135.0  
         660.0   135.0  
         720.0   135.0  
         780.0   135.0  

[4942355 rows x 14 columns]

In [ ]:
len(pd.unique(timeseries_lab.index.get_level_values(0)))

72184

In [ ]:
final = final.merge(timeseries_lab, left_index=True, right_index=True)

In [ ]:
final

PAD  HF  AF  ARR  unitdischargeoffset  \
patient time                                             
144989  -660.0     0   0   0    0                 2760   
        -600.0     0   0   0    0                 2760   
        -540.0     0   0   0    0                 2760   
        -480.0     0   0   0    0                 2760   
        -420.0     0   0   0    0                 2760   
...              ...  ..  ..  ...                  ...   
3352962  1440.0    0   0   0    0                 2241   
         1500.0    0   0   0    0                 2241   
         1560.0    0   0   0    0                 2241   
         1620.0    0   0   0    0                 2241   
         1680.0    0   0   0    0                 2241   

                 predictedhospitalmortality  actualhospitalmortality  gender  \
patient time                                                                   
144989  -660.0                     0.092861                        0     0.0   
        -600.0                     0.092861                        0     0.0   
        -540.0                     0.092861                        0     0.0   
        -480.0                     0.092861                        0     0.0   
        -420.0                     0.092861                        0     0.0   
...                                     ...                      ...     ...   
3352962  1440.0                    0.030529                        0     1.0   
         1500.0                    0.030529                        0     1.0   
         1560.0                    0.030529                        0     1.0   
         1620.0                    0.030529                        0     1.0   
         1680.0                    0.030529                        0     1.0   

                  age  admissionheight  admissionweight  hour  teachingstatus  \
patient time                                                                    
144989  -660.0   78.0            158.0             60.5     3               1   
        -600.0   78.0            158.0             60.5     3               1   
        -540.0   78.0            158.0             60.5     3               1   
        -480.0   78.0            158.0             60.5     3               1   
        -420.0   78.0            158.0             60.5     3               1   
...               ...              ...              ...   ...             ...   
3352962  1440.0  60.0            182.9             94.5     7               0   
         1500.0  60.0            182.9             94.5     7               0   
         1560.0  60.0            182.9             94.5     7               0   
         1620.0  60.0            182.9             94.5     7               0   
         1680.0  60.0            182.9             94.5     7               0   

                 ethnicity_African American  ethnicity_Asian  \
patient time                                                   
144989  -660.0                            0                0   
        -600.0                            0                0   
        -540.0                            0                0   
        -480.0                            0                0   
        -420.0                            0                0   
...                                     ...              ...   
3352962  1440.0                           0                0   
         1500.0                           0                0   
         1560.0                           0                0   
         1620.0                           0                0   
         1680.0                           0                0   

                 ethnicity_Caucasian  ethnicity_Hispanic  \
patient time                                               
144989  -660.0                     1                   0   
        -600.0                     1                   0   
        -540.0                     1                   0   
        -480.0                    

In [ ]:
timeseries_aperiodic = pd.read_csv('timeseries_aperiodic.csv', index_col=['patient', 'time'])

In [ ]:
timeseries_aperiodic

noninvasivesystolic  noninvasivediastolic  \
patient time                                                         
141168  0 days 02:03:00                106.0                  68.0   
        0 days 02:18:00                111.0                  62.0   
        0 days 05:49:00                  NaN                   NaN   
        0 days 07:21:00                  NaN                   NaN   
        0 days 23:18:00                  NaN                   NaN   
...                                      ...                   ...   
3353263 0 days 11:50:00                118.0                  85.0   
        0 days 12:50:00                128.0                  91.0   
        0 days 13:50:00                131.0                  88.0   
        0 days 14:50:00                147.0                  98.0   
        0 days 15:19:00                144.0                  95.0   

                         noninvasivemean  
patient time                              
141168  0 days 02:03:00             81.0  
        0 days 02:18:00             82.0  
        0 days 05:49:00             79.0  
        0 days 07:21:00             62.0  
        0 days 23:18:00             27.0  
...                                  ...  
3353263 0 days 11:50:00             98.0  
        0 days 12:50:00            106.0  
        0 days 13:50:00            107.0  
        0 days 14:50:00            118.0  
        0 days 15:19:00            114.0  

[20987170 rows x 3 columns]

In [ ]:
# take the mean of any duplicate index entries for unstacking
timeseries_aperiodic = timeseries_aperiodic.groupby(level=[0, 1]).mean()

# Round up the time-stamps to the next hour
timeseries_aperiodic.reset_index(level=1, inplace=True)
start = pd.to_datetime('2000-01-01 00:00:00')
timeseries_aperiodic.time = pd.to_timedelta(timeseries_aperiodic.time, errors='coerce') + start
timeseries_aperiodic.time = timeseries_aperiodic.time.dt.ceil(freq='H')
timeseries_aperiodic.time = timeseries_aperiodic.time - start
timeseries_aperiodic.time = pd.to_timedelta(timeseries_aperiodic.time, unit='T')
timeseries_aperiodic.set_index('time', append=True, inplace=True)
timeseries_aperiodic.reset_index(level=0, inplace=True)
resampled = timeseries_aperiodic.groupby('patient').resample('H', closed='right', label='right').mean().drop(columns='patient')

In [ ]:
resampled

noninvasivesystolic  noninvasivediastolic  \
patient time                                                         
141168  0 days 03:00:00                108.5                  65.0   
        0 days 04:00:00                  NaN                   NaN   
        0 days 05:00:00                  NaN                   NaN   
        0 days 06:00:00                  NaN                   NaN   
        0 days 07:00:00                  NaN                   NaN   
...                                      ...                   ...   
3353263 0 days 13:00:00                128.0                  91.0   
        0 days 14:00:00                131.0                  88.0   
        0 days 15:00:00                147.0                  98.0   
        0 days 16:00:00                144.0                  95.0   
        0 days 17:00:00                  NaN                   NaN   

                         noninvasivemean  
patient time                              
141168  0 days 03:00:00             81.5  
        0 days 04:00:00              NaN  
        0 days 05:00:00              NaN  
        0 days 06:00:00             79.0  
        0 days 07:00:00              NaN  
...                                  ...  
3353263 0 days 13:00:00            106.0  
        0 days 14:00:00            107.0  
        0 days 15:00:00            118.0  
        0 days 16:00:00            114.0  
        0 days 17:00:00              NaN  

[10243399 rows x 3 columns]

In [ ]:
resampled.update(resampled.groupby(level=0).ffill())

In [ ]:
timeseries_aperiodic = resampled

In [ ]:
timeseries_aperiodic.reset_index(level=1, inplace=True)
timeseries_aperiodic.time = pd.to_timedelta(timeseries_aperiodic.time, errors='coerce')

# Convert the time-stamps into minutes
timeseries_aperiodic.time = timeseries_aperiodic.time.astype(int)/(1000000000*60)

timeseries_aperiodic.reset_index(inplace=True)
timeseries_aperiodic.set_index(['patient', 'time'], inplace=True)

In [ ]:
timeseries_aperiodic

noninvasivesystolic  noninvasivediastolic  noninvasivemean
patient time                                                              
141168  180.0                 108.5                  65.0             81.5
        240.0                 108.5                  65.0             81.5
        300.0                 108.5                  65.0             81.5
        360.0                 108.5                  65.0             79.0
        420.0                 108.5                  65.0             79.0
...                             ...                   ...              ...
3353263 780.0                 128.0                  91.0            106.0
        840.0                 131.0                  88.0            107.0
        900.0                 147.0                  98.0            118.0
        960.0                 144.0                  95.0            114.0
        1020.0                144.0                  95.0            114.0

[10243399 rows x 3 columns]

In [ ]:
pd.set_option('display.max_columns', 1000)  # or 1000
pd.set_option('display.max_rows', 1000)  # or 1000
pd.set_option('display.max_colwidth', 1000)  # or 199

# Missing values per feature
timeseries_aperiodic.isnull().sum(axis = 0)/10243399*100

noninvasivesystolic     0.345823
noninvasivediastolic    0.344866
noninvasivemean         0.332653
dtype: float64

In [ ]:
len(pd.unique(timeseries_aperiodic.index.get_level_values(0)))

145754

In [ ]:
# Drop missing values samples
missing_samples = timeseries_aperiodic[timeseries_aperiodic.isnull().any(axis=1)].index.get_level_values(0).tolist()
timeseries_aperiodic.drop(missing_samples, level=0, axis=0, inplace=True)

In [ ]:
final = final.merge(timeseries_aperiodic, left_index=True, right_index=True)

In [ ]:
final

PAD  HF  AF  ARR  unitdischargeoffset  \
patient time                                            
144989  60.0      0   0   0    0                 2760   
        120.0     0   0   0    0                 2760   
        180.0     0   0   0    0                 2760   
        240.0     0   0   0    0                 2760   
        300.0     0   0   0    0                 2760   
...             ...  ..  ..  ...                  ...   
3352962 1440.0    0   0   0    0                 2241   
        1500.0    0   0   0    0                 2241   
        1560.0    0   0   0    0                 2241   
        1620.0    0   0   0    0                 2241   
        1680.0    0   0   0    0                 2241   

                predictedhospitalmortality  actualhospitalmortality  gender  \
patient time                                                                  
144989  60.0                      0.092861                        0     0.0   
        120.0                     0.092861                        0     0.0   
        180.0                     0.092861                        0     0.0   
        240.0                     0.092861                        0     0.0   
        300.0                     0.092861                        0     0.0   
...                                    ...                      ...     ...   
3352962 1440.0                    0.030529                        0     1.0   
        1500.0                    0.030529                        0     1.0   
        1560.0                    0.030529                        0     1.0   
        1620.0                    0.030529                        0     1.0   
        1680.0                    0.030529                        0     1.0   

                 age  admissionheight  admissionweight  hour  teachingstatus  \
patient time                                                                   
144989  60.0    78.0            158.0             60.5     3               1   
        120.0   78.0            158.0             60.5     3               1   
        180.0   78.0            158.0             60.5     3               1   
        240.0   78.0            158.0             60.5     3               1   
        300.0   78.0            158.0             60.5     3               1   
...              ...              ...              ...   ...             ...   
3352962 1440.0  60.0            182.9             94.5     7               0   
        1500.0  60.0            182.9             94.5     7               0   
        1560.0  60.0            182.9             94.5     7               0   
        1620.0  60.0            182.9             94.5     7               0   
        1680.0  60.0            182.9             94.5     7               0   

                ethnicity_African American  ethnicity_Asian  \
patient time                                                  
144989  60.0                             0                0   
        120.0                            0                0   
        180.0                            0                0   
        240.0                            0                0   
        300.0                            0                0   
...                                    ...              ...   
3352962 1440.0                           0                0   
        1500.0                           0                0   
        1560.0                           0                0   
        1620.0                           0                0   
        1680.0                           0                0   

                ethnicity_Caucasian  ethnicity_Hispanic  \
patient time                                              
144989  60.0                      1                   0   
        120.0                     1                   0   
        180.0                     1                   0   
        240.0                     1                   0   
        300.0                  

In [ ]:
(final < 0)

PAD     HF     AF    ARR  unitdischargeoffset  \
patient time                                                      
144989  60.0    False  False  False  False                False   
        120.0   False  False  False  False                False   
        180.0   False  False  False  False                False   
        240.0   False  False  False  False                False   
        300.0   False  False  False  False                False   
...               ...    ...    ...    ...                  ...   
3352962 1440.0  False  False  False  False                False   
        1500.0  False  False  False  False                False   
        1560.0  False  False  False  False                False   
        1620.0  False  False  False  False                False   
        1680.0  False  False  False  False                False   

                predictedhospitalmortality  actualhospitalmortality  gender  \
patient time                                                                  
144989  60.0                         False                    False   False   
        120.0                        False                    False   False   
        180.0                        False                    False   False   
        240.0                        False                    False   False   
        300.0                        False                    False   False   
...                                    ...                      ...     ...   
3352962 1440.0                       False                    False   False   
        1500.0                       False                    False   False   
        1560.0                       False                    False   False   
        1620.0                       False                    False   False   
        1680.0                       False                    False   False   

                  age  admissionheight  admissionweight   hour  \
patient time                                                     
144989  60.0    False            False            False  False   
        120.0   False            False            False  False   
        180.0   False            False            False  False   
        240.0   False            False            False  False   
        300.0   False            False            False  False   
...               ...              ...              ...    ...   
3352962 1440.0  False            False            False  False   
        1500.0  False            False            False  False   
        1560.0  False            False            False  False   
        1620.0  False            False            False  False   
        1680.0  False            False            False  False   

                teachingstatus  ethnicity_African American  ethnicity_Asian  \
patient time                                                                  
144989  60.0             False                       False            False   
        120.0            False                       False            False   
        180.0            False                       False            False   
        240.0            False                       False            False   
        300.0            False                       False            False   
...                        ...                         ...              ...   
3352962 1440.0           False                       False            False   
        1500.0           False                       False            False   
        1560.0           False                       False            False   
        1620.0           False                       False            False   
        1680.0           False                       False            False   

                ethnicity_Caucasian  ethnicity_Hispanic  \
patient time                                              
144989  60.0                  False               False   
        120.0                 False               Fals

In [ ]:
(final < 0).apply(lambda x: any(x))

PAD                                                 False
HF                                                  False
AF                                                  False
ARR                                                 False
unitdischargeoffset                                 False
predictedhospitalmortality                           True
actualhospitalmortality                             False
gender                                              False
age                                                 False
admissionheight                                     False
admissionweight                                     False
hour                                                False
teachingstatus                                      False
ethnicity_African American                          False
ethnicity_Asian                                     False
ethnicity_Caucasian                                 False
ethnicity_Hispanic                                  False
ethnicity_Nati

In [ ]:
final.loc[final['predictedhospitalmortality'] <= 0]

PAD  HF  AF  ARR  unitdischargeoffset  \
patient time                                             
146709  120.0      0   0   1    1                15993   
        180.0      0   0   1    1                15993   
        240.0      0   0   1    1                15993   
        300.0      0   0   1    1                15993   
        360.0      0   0   1    1                15993   
...              ...  ..  ..  ...                  ...   
3348321 16680.0    0   1   0    0                17917   
        16740.0    0   1   0    0                17917   
        16800.0    0   1   0    0                17917   
        16860.0    0   1   0    0                17917   
        16920.0    0   1   0    0                17917   

                 predictedhospitalmortality  actualhospitalmortality  gender  \
patient time                                                                   
146709  120.0                          -1.0                        0     0.0   
        180.0                          -1.0                        0     0.0   
        240.0                          -1.0                        0     0.0   
        300.0                          -1.0                        0     0.0   
        360.0                          -1.0                        0     0.0   
...                                     ...                      ...     ...   
3348321 16680.0                        -1.0                        1     0.0   
        16740.0                        -1.0                        1     0.0   
        16800.0                        -1.0                        1     0.0   
        16860.0                        -1.0                        1     0.0   
        16920.0                        -1.0                        1     0.0   

                  age  admissionheight  admissionweight  hour  teachingstatus  \
patient time                                                                    
146709  120.0    61.0            172.7            139.0    17               1   
        180.0    61.0            172.7            139.0    17               1   
        240.0    61.0            172.7            139.0    17               1   
        300.0    61.0            172.7            139.0    17               1   
        360.0    61.0            172.7            139.0    17               1   
...               ...              ...              ...   ...             ...   
3348321 16680.0  67.0            172.7            108.0     3               0   
        16740.0  67.0            172.7            108.0     3               0   
        16800.0  67.0            172.7            108.0     3               0   
        16860.0  67.0            172.7            108.0     3               0   
        16920.0  67.0            172.7            108.0     3               0   

                 ethnicity_African American  ethnicity_Asian  \
patient time                                                   
146709  120.0                             0                0   
        180.0                             0                0   
        240.0                             0                0   
        300.0                             0                0   
        360.0                             0                0   
...                                     ...              ...   
3348321 16680.0                           1                0   
        16740.0                           1                0   
        16800.0                           1                0   
        16860.0                           1                0   
        16920.0                           1                0   

                 ethnicity_Caucasian  ethnicity_Hispanic  \
patient time                                               
146709  120.0                      1                   0   
        180.0                      1                   0   
        240.0                      1                   0   
        300.0                     

In [ ]:
final.predictedhospitalmortality.replace([-1], [0], inplace=True)

In [ ]:
final.columns

Index(['PAD', 'HF', 'AF', 'ARR', 'unitdischargeoffset',
       'predictedhospitalmortality', 'actualhospitalmortality', 'gender',
       'age', 'admissionheight', 'admissionweight', 'hour', 'teachingstatus',
       'ethnicity_African American', 'ethnicity_Asian', 'ethnicity_Caucasian',
       'ethnicity_Hispanic', 'ethnicity_Native American',
       'ethnicity_Other/Unknown', 'unittype_CCU-CTICU', 'unittype_CSICU',
       'unittype_CTICU', 'unittype_Cardiac ICU', 'unittype_MICU',
       'unittype_Med-Surg ICU', 'unittype_Neuro ICU', 'unittype_SICU',
       'unitadmitsource_Acute Care/Floor', 'unitadmitsource_Direct Admit',
       'unitadmitsource_Emergency Department', 'unitadmitsource_Floor',
       'unitadmitsource_Operating Room', 'unitadmitsource_Other Hospital',
       'unitadmitsource_PACU', 'unitadmitsource_Recovery Room',
       'unitadmitsource_Step-Down Unit (SDU)', 'unitadmitsource_misc',
       'unitvisitnumber_1', 'unitvisitnumber_2', 'unitvisitnumber_3',
       'unitvisit

In [ ]:
len(pd.unique(final.index.get_level_values(0)))

1433

In [ ]:
!pip install scikit-multilearn
from skmultilearn.model_selection import iterative_train_test_split

In [ ]:
y = final[['actualhospitalmortality', 'PAD', 'HF', 'ARR', 'AF']]
y = y.droplevel(1)
y = y[~y.index.duplicated(keep='first')]

In [ ]:
X = final.drop(labels=['unitdischargeoffset', 'actualhospitalmortality', 'PAD', 'HF', 'ARR', 'AF'], axis=1, inplace=False)

In [ ]:
X_ind = X.index.get_level_values(0)
X_ind = X_ind[~X_ind.duplicated(keep='first')]
X_ind = np.tile(X_ind, (2, 1))
X_ind = X_ind.transpose()

In [ ]:
# Split the data into train and test

y = y.to_numpy()

X_train_ind, y_train, X_test_ind, y_test = iterative_train_test_split(X_ind, y, test_size = 0.2)
X_train_ind, y_train, X_val_ind, y_val = iterative_train_test_split(X_train_ind, y_train, test_size = 0.2)

In [ ]:
X.reset_index(level=1, inplace=True)
X_train = X.loc[X_train_ind[:, 0].tolist()]
X_val = X.loc[X_val_ind[:, 0].tolist()]
X_test = X.loc[X_test_ind[:, 0].tolist()]

In [ ]:
X_train.reset_index(inplace=True)
X_val.reset_index(inplace=True)
X_test.reset_index(inplace=True)

X_train.set_index(['patient', 'time'], inplace=True)
X_val.set_index(['patient', 'time'], inplace=True)
X_test.set_index(['patient', 'time'], inplace=True)

In [ ]:
# create time-series input for LSTM of shape [n, timestep, features]
def split_sequence(dataframe, n_steps):
    lstm_input = np.empty((len(dataframe.index.levels[0]), n_steps, 80))
    lstm_input[:] = np.nan
    for i in range(len(dataframe.index.levels[0])):
        sample = dataframe.loc[dataframe.index.levels[0][i].tolist()]
        sequence = sample.to_numpy()
        n_features = sequence.shape[1]
        time_length = sequence.shape[0]

        if n_steps > time_length:
            a = np.empty((n_steps-time_length,n_features))
            for j in range((n_steps-time_length)):
                a[j, :] = sequence[0, :]
            sequence = np.vstack((a,sequence))
        else: sequence = sequence[-n_steps:, :]
        lstm_input[i, :, :] = sequence

    return lstm_input

In [ ]:
# Extract data into LSTM timeseries format with 24 1-hour timesteps
X_train = split_sequence(X_train, 24)
X_val = split_sequence(X_val, 24)
X_test = split_sequence(X_test, 24)

In [ ]:
X_test.shape

(287, 24, 80)

In [ ]:
np.isnan(X_test).any()

False

In [ ]:
np.any(X_train < 0)

False

In [ ]:
np.save('X_train_multilabel_full_42', X_train)
np.save('X_val_multilabel_full_42', X_val)
np.save('X_test_multilabel_full_42', X_test)
np.save('y_train_multilabel_full_42', y_train)
np.save('y_val_multilabel_full_42', y_val)
np.save('y_test_multilabel_full_42', y_test)